In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_json("2vr3-k9wn.json")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe(include='all', percentiles=[.05, .25, .5, .75, 0.95])
#50 percentile = median

notice something wrong:
* "h_mag" we have only 181/202 continuous values, missing 11
* "period_yr" has 200/202 values, missing 2
* "q_au_2" has 200/202, probably 2 values wrong

the other columns with categorical data:
* designation: how many unique values ? sometimes it holds date data, to be cleaned, 202/202 ok
* discovery_date: object, probably string type, needs to be datetime, split into years and month and day columns, 202/202 ok
* orbit_class: str 202/202 looks good, perhaps transform into dummy columns for numerical analysis if needed
* pha: binary values, str type 'Y' or 'N', maybe replace with binary value N=0 and Y=1

In [ ]:
#create a variable
h_mag = df["h_mag"]

In [ ]:
#check all unique values in this column, notice 'nan', could be null or string, must be cleaned
#we could choose to lower the data type from float64 to int and lower the storage space
h_mag.unique()

In [ ]:
h_mag[ h_mag.isnull() ]
#these are all the records with null values, (21x values, 10% that's a lot !)
#what we could try is fill in with dummy values based on mean values

In [ ]:
plt.figure(figsize=(12,8))
h_mag.plot.hist(bins=len(h_mag))
plt.plot()

#this shows some sort of a normal distribution, with a mean=20.3 and a standard deviation=1.52
#the range = 24.3 - 15.6 = 8,7
#let's check the 

In [ ]:
np.mean(h_mag[h_mag.notnull()])

In [ ]:
np.median(h_mag[h_mag.notnull()])

In [ ]:
#this is GOOD ! this means that we can use 20.3 as a default value to replace the NULL values

In [ ]:
h_mag[ h_mag.isnull() ] = float(20.3)

In [ ]:
#let's work on "period_yr"
period_yr = df["period_yr"]

In [ ]:
period_yr.unique()
# again some nan values
# some extremely large numbers, lets check

In [ ]:
period_yr[ period_yr.isnull() ]
#ok so only 2x nan values

In [ ]:
#between Q1 and Q3 : 1.85 and 4.39
#so, what are the values inside this ideal IQR range ? 
period_yr[ (period_yr > 1.855000e+00) & (period_yr < 4.390000e+00) ]
#Maybe we need a z-score for this...

In [ ]:
x = period_yr[ (period_yr > 9.405000e-01) & (period_yr < 2.354150e+01) ]
plt.figure(figsize=(12,8))
x.plot.hist(bins=len(x))
plt.plot()

In [ ]:
np.mean(x)

In [ ]:
np.median(x)

In [ ]:
#quite a uniform distribution, we can replace the 2 nan values with the median value 3.25
period_yr[ period_yr.isnull() ] = float(3.25)

In [ ]:
period_yr[147]

In [ ]:
x = period_yr[ (period_yr > 0) & (period_yr < 10) ]
plt.figure(figsize=(12,8))
x.plot.hist(bins=len(x))
plt.plot()

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(x, bins=40)
#I think if we replace the outlier values with these median values that the distribution becomes more normal,
#but I am afraid this would damage the truth these numbers are telling, maybe a sigma 2 or 3 cutoff would be better...
#don't know how to do or if important

In [ ]:
#ok so now we should try to clean "q_au_1"
q_au_1 = df["q_au_1"]
q_au_2 = df["q_au_2"]

In [ ]:
q_au_1.unique()
# looking rather normal

In [ ]:
q_au_2.unique()

# this should be cleaned...

In [ ]:
#show only the values between the 5% and 95% range, erase the outliers
x = q_au_2[ (q_au_2 > 1.228000) & (q_au_2 < 16.947000) ]
plt.figure(figsize=(12,8))
x.plot.hist(bins=len(x))
plt.plot()

In [ ]:
q_au_2[ q_au_2.isnull() ]
#ok so only 2x nan values

In [ ]:
np.median(x)

In [ ]:
#quite a uniform distribution, we can replace the 2 nan values with the median value 3.25
q_au_2[ q_au_2.isnull() ] = float(3.47)

In [ ]:
#leaving the outliers... 

In [ ]:
designation = df["designation"]

In [ ]:
designation.nunique()
# we have 202 unique observations/labels

In [ ]:
designation[designation.unique()]
# we should clean up this data, remove the weird number, remove the date number, and simply keep the code like "AH37"
# as a string value, we can try to use regular expressions for this

In [ ]:
text = designation.unique()[0]
print (text)
#let's exercise cleaning this one thing...

In [ ]:
import re
#loading regex / regular expression tools
#also go to https://regex101.com/  for getting the right code
#check out : https://en.wikipedia.org/wiki/Provisional_designation_in_astronomy   to figure out the letter combinations

In [ ]:
"""
example text:

419880 (2011 AH37)
(2015 HF11)
C/2010 E3 (WISE)
(2010 CP140)

[ABCDEFGHJKLMNOPQRSTUVWXY][ABCDEFGHJKLMNOPQRSTUVWXYZ]?[\d]+
"""

In [ ]:
regex = r"[ABCDEFGHJKLMNOPQRSTUVWXY][ABCDEFGHJKLMNOPQRSTUVWXYZ]?[\d]+"

In [ ]:
test_str = ("419880 (2011 AH37)\n"
            "(2015 HF11)\n"
            "C/2010 E3 (WISE)\n"
            "(2010 CP140)")

In [ ]:
re.findall(regex, test_str)

In [ ]:
#ok, so this is proove we can extract this content and now we need to create a function to automate this in the dataset
#if we replace the test_str with the "designation" data

In [ ]:
test_str = designation.values
test_str

In [ ]:
for test_str in designation:
    print (test_str)
    print (re.findall(regex, test_str))
    #if we verify this list we can see mistakes that we need to resolve
    #if empty fill in original value or do another regex translation...

In [ ]:
from datetime import datetime

In [ ]:
discovery_date = df["discovery_date"]

In [ ]:
discovery_date[0]

In [ ]:
discovery_date[0].split("T")

In [ ]:
one_date = discovery_date[0].split("T")[0]
one_date

In [ ]:
one_date = datetime.strptime(one_date, "%Y-%m-%d")
one_date

In [ ]:
print (one_date.year)
print (one_date.month)
print (one_date.day)

In [ ]:
#now we update the dataframe with the right columns 

In [ ]:
df["discovery_date"] = discovery_date.apply(lambda x: x.split("T")[0])
df["discovery_date"]

In [ ]:
df["discovery_year"] = df["discovery_date"].apply(lambda x: int(x.split("-")[0]))

In [ ]:
df["discovery_month"] = df["discovery_date"].apply(lambda x: int(x.split("-")[1]))

In [ ]:
df["discovery_day"] = df["discovery_date"].apply(lambda x: int(x.split("-")[2]))

In [ ]:
df.drop(['discovery_date'], axis=1, inplace=True)
#removing the original date string

In [ ]:
#... work in progress, quite fun... could all be worthless  ! :-)

In [ ]:
"""
http://4cminews.com/?p=30879
https://en.wikipedia.org/wiki/2016_WF9

designation    = year + identifier of the object in space (ex: 2016 WF9) - wikipedia.org/wiki/2016_WF9
discovery_date = the date the NEOWISE space RADAR detected the object (discovery: first observed) yyyy-mm-dd
h_mag          = absolute magnitude of the object in log scale (ex: H (mag)= 20.1) - wikipedia.org/wiki/Absolute_magnitude
               = only objects larger than roughly 140 meters in diameter (or absolute magnitude, H > 22)
               = see : https://en.wikipedia.org/wiki/Minimum_orbit_intersection_distance
i_deg          = inclination degree of the orbit (irrelevant), the tilt of the object orbit around a body
moid_au        = minimum orbit intersection distance
               = An object is classified as a potentially hazardous object (PHO) – that is,posing a possible risk to Earth –
               = if, among other conditions, its Earth MOID is less than 0.05 AU.
               = (MOID < 0.05)
               = https://en.wikipedia.org/wiki/Minimum_orbit_intersection_distance
               = ex: earth MOID for 2016_WF9 = 0.0156 AU ~ MOID (au): 0.015 in dataset
orbit_class    = group name of the Near Earth Orbit route around the sun compared to earth (irrelevant)
period_yr      = how many earth years the object makes it's own full orbit
               = the time a given astronomical object takes to complete one orbit around another object
               = in degrees °, ex:  Inclination	14.995°  or i (deg): 15 (irrelevant)
pha            = potentially hazardous asteroids (Y/N) binary data
               = suspected extinct comet, classified as near-Earth object and potentially hazardous asteroid of Apollo group
q_au_1         = (AU) min amplitude (= earth is at 1 AU)
               = q (au): 0.98 or Perihelion  0.9816 AU
               = the point where the body comes closest to the Sun
q_au_2         = (AU) max amplitude (= earth is at 1 AU)
               = Q (au): 4.76 or Aphelion  4.7614 AU
               = which is the point in the orbit where the celestial body is farthest from the Sun
               = https://en.wikipedia.org/wiki/Perihelion_and_aphelion
"""